# Algorithme du simplexe: méthode des dictionnaires

Le but de ce TP est d'implémenter la méthode des dictionnaires, une implémentation de l'algorithme du simplexe.

L'algorithme du simplexe permet de résoudre les problèmes de programmation linéaire sous forme standard c'est à dire
$$
\min_{x \in \mathbb{R}^n | A x = b} \langle c, x \rangle 
$$
avec :
- $x, c \in \mathbb{R}^n$
- $b \in \mathbb{R}^m$
- $A \in \mathcal{M}_{m \times n}(\mathbb{R})$ avec $m \leq n$ et $A$ de rang $m$.

In [ ]:
import numpy as np

## I. Implémentation des dictionnaires

Pour rappel, un dictionnaire est une structure permettant de manipuler les iterations de l'algorithme du simplexe. Pour un itéré de base $K$ le dictionnaire associé est de la forme
$$
\begin{array}{c|c}
 -\langle c_K, A_K^{-1} b \rangle & r^T\\\hline
 x_K = A_K^{-1} b & A_K^{-1} A
\end{array}
$$
où $r^T = c^T - c_K^T A_K^{-1} A$ est le vecteur des coûts réduits.

Afin de manipuler les dictionnaires, nous allons utiliser la structure `Dictionary` définie ci dessous. 
Elle contient les champs:
- `m` : le nombre de contraintes
- `n` : le nombre de variables
- `c` : un `np.array` définissant de taille $n$ la fonction coût
- `labels` : une `list` de `str` permettant de nommer les `n` variables
- `basis` : les indices des `m` variables dans la base
- `D` : un `np.array` de taille $(m+1) \times (n+1)$ dictionnaire 

Attention, il est important de bien utiliser les types définis pour ces variables, les fonctions suivantes de ce TP pourrait mal se comporter sinon.
En toute rigeur, il faudrait définir des méthodes qui permettent de mettre à jour les attributs de la classe et qui vérifient le bon typage des variables. Mais cela sort du cadre de ce TP.

La classe contient une méthode `display()` permettant d'afficher le dictionnaire.

In [ ]:
class Dictionary():
    def __init__(self, m, n):
        self.m = m # number of constraints
        self.n = n # number of variables+
        self.D = np.zeros((m+1,n+1)) # dictionary
        self.c = np.zeros(n) # cost function vector
        self.labels = {} # labels of the variables
        self.basic = np.zeros(m) # which variables are basic

    # this method displays the dictionary
    def display(self):
        """
        Display this simplex dictionary on the standard Jupyter output.

        Args:
            name: Name of the dictionary.
        """
        from IPython.display import Math, display

        d = (
            r"\begin{{array}}{{r|{}}}".format("r" * (self.D.shape[1]))
            + r"\hline "
            + np.array2string(self.D[0,:], separator=" & ", formatter={'float_kind':'{:f}'.format})[1:-1]
            + r"\\\hline "
            + r"\\".join(
                (self.labels[self.basic[i-1]] + " = " if len(self.labels) == self.n else "")
                +
                np.array2string(self.D[i,:], separator= " & ", formatter={'float_kind':'{:f}'.format})[1:-1]
                for i in range(1,self.D.shape[0])
                )
            + r"\\\hline\end{array}"
        )

        display(Math(d))
        x = np.zeros(self.n)
        x[self.basic] = self.D[1:,0]
        print('Current point ' + np.array2string(x, separator= ", "))
        print('Current cost : %.2f' % (-self.D[0,0]))

Par exemple, le problème linéaire sous forme standard suivant :
$$
\begin{array}{rrrlrlrlrlrlrl}
\max & z = & 3 x_1 & + & 2 x_2 &   &     &   &    \\
s.c.     & & 2 x_1 & + &   x_2 & + & x_3 & = & 18  \\
         & & 2 x_1 & + & 3 x_2 & + & x_4 & = & 42 \\
         & & 3 x_1 & + &   x_2 & + & x_5 & = & 24 \\
         & & x_j & \geq & 0 & \quad & j = & 1, & 2, & 3, & 4, & 5
\end{array}
$$
possède une base évidente composée des variables d'écart $K = (x_3, x_4, x_5)$. Il est ainsi possible de construire le dictionaire suivant:
$$
\begin{array}{l|l l l l l|}
 0 & -3 & -2 & 0 & 0 & 0 \\\hline
 x_3 = 18 & 2 & 1 & 1 & 0 & 0\\
 x_4 = 42 & 2 & 3 & 0 & 1 & 0 \\
 x_5 = 24 & 3 & 1 & 0 & 0 & 1 \\\hline
\end{array}
$$
car comme $x_1 = x_2 = 0$ les varialbes d'écarts $x_3 = b_1$, $x_4 = b_2$, $x_5 = b_3$. De plus comme $c_K = 0$, on obtient  $\langle c_K, A_K^{-1} b \rangle  = 0$ et $r^T = c^T$.

Le code ci-dessous permet de créer le dictionnaire associé. 

In [ ]:
n = 2 # nombre de variables
m = 3 # nombre de contraintes

dict =  Dictionary(m,n+m) # taille du dictionnaire avec les variables d'écarts

dict.D[0,0] = 0 # coût à l'itéré courant -\langle c_K, A_K^{-1} b \rangle dans la première case du tableau
dict.D[1:,0] = [18, 42, 24] # valeurs x_K des variables de base dans la première colonne 
dict.D[0,1:] = [-3, -2, 0, 0, 0] # vecteur des coûts réduits r^T dans la première ligne
dict.D[1:,1:] = [[2, 1, 1, 0, 0], [2, 3,0,1,0], [3,1,0,0,1]] # matrice des contraintes A_K^{-1} A dans le reste du tableau 
dict.basic = [2,3,4] # les variables de bases sont d'indices 2,3,4 
dict.c[:] = [-3,-2,0,0,0] # le vecteur définissant la fonction cout
dict.labels = list(['x_1', 'x_2', 'x_3', 'x_4', 'x_5']) # le nom des variables

dict.display()

**Todo:** Etudier le code pour comprendre la structure du dictionnaire et l'organisation de `dict.D`.

## II. Implémentation de la méthode du simplexe

Dans cette partie, nous allons implémenter l'algorithme du simplexe qui manipulera la structure `Dictionary`. La règle de Bland sera utilisée (règle de plus petit indice) afin de selectionner les variables entrantes et sortantes.

**Todo:** Pour le dictionnaire précédent, c'est à dire 
$$
\begin{array}{l|l l l l l|}
 0 & -3 & -2 & 0 & 0 & 0 \\\hline
 x_3 = 18 & 2 & 1 & 1 & 0 & 0\\
 x_4 = 42 & 2 & 3 & 0 & 1 & 0 \\
 x_5 = 24 & 3 & 1 & 0 & 0 & 1 \\\hline
\end{array}
$$
trouver la variable entrante, la variable sortante et effectuer une étape de l'algorithme du simplexe. 
Réiterer jusqu'à convergence.
Ceci permettra de tester vos fonctions.

### 1. Recherche des variables entrantes et sortantes

Ecrire une fonction `find_entering_variable` permettant de trouver la variable entrante d'un dictionnaire donnée. Cette fonction prend en argument:
- `D` : le `np.array` de taille $(m+1, n+1)$ représentant le dictionnaire.

Et retourne
- `index_entering` : l'indice de la variable entrant le dictionnaire. C'est à dire, le plus petit indice des variables pour lesquelles les coûts réduits sont strictement négatifs. Si tous les coûts réduits sont positifs ou nuls, la fonction devra retourner `index_entering = None`

A cette fin, vous pouvez utiliser une boucle sur les coûts réduits (c'est à dire `D[0,1:]`) qui s'arrete dès qu'elle recontre un cout réduit négatif et retourne le dernière indice de l'itération. 

Attention, cette fonction doit retourner un indice entre $0$ et $n-1$. En fonction de votre implémentation, vérifier que l'indice retourné n'est pas décalé de 1. Ceci sera important pour les fonctions prédéfinies dans les sections suivantes.


In [ ]:
def find_entering_variable(D):
    # à coder
    return index_entering 

**Test:** Vérifier que la fonction retourne bien la bonne variable pour le dictionnaire `dict` défini.

In [ ]:
index_entering = find_entering_variable(dict.D)
print(index_entering)

Ecrire une fonction `find_leaving_variable` permettant de trouver la variable sortante pour un dictionnaire et une variable entrante donnés. Cette fonction prend en argument :
- `D` : le `np.array` de taille $(m+1, n+1)$ représentant le dictionnaire.
- `index_entering` : l'indice de la variable entrant le dictionnaire. Si aucune variable ne peut entrer la base la fonction doit retourner `index_entering = None`

Elle retourne `index_leaving` l'indice de la variable sortant dans la base. C'est à dire que si la base est $K = \{ x_4, x_2, x_1 \}$ et que $x_4$ sort de la base, `index_leaving = 0`. De même, si $x_1$ sortait de la base on aurait `index_leaving = 2`.
Notons que l'indice de la variable dans le vecteur $x$ peut être retrouvé en utilisant `dict.basic[index_leaving]`. Dans l'exemple précédent on aurait en effet `dict.basic = [3,1,0]`.


Notons $u$ la direction de descente admissible associée à la variable entrante, c'est à dire `D[1:,index_entering]`. Si toutes les composantes de $u$ sont négatives ou nulles, aucune variable ne peut entrer la base (car le problème est non borné).
Dans le cas contrainte, la variable entrant la base sera d'indice réalisant le minimum :
$$
    \min_{i | u_i > 0} \frac{x_i}{u_i}.
$$
A cet fin, vous pouvez également utiliser une boucle calculant les ratios $x_i / u_i$ pour les $u_i > 0$ et qui retient le minimum et l'indice le réalisant.
Similairement `index_leaving` doit être entre $0$ et $m-1$.

In [ ]:
def find_leaving_variable(D,index_entering):
    ## à coder
    return index_leaving

**Test:** Vérifier que la fonction retourne bien la variable sortante pour le dictionnaire défini.

In [ ]:
index_entering = find_entering_variable(dict.D)
index_leaving = find_leaving_variable(dict.D, index_entering)
print(index_entering)
print(index_leaving)
print(dict.basic[index_leaving])

### 2. Pivot du dictionnaire

La prochaine fonction à implémenter va réaliser l'étape de pivot du dictionnaire à partir d'une variable entrante et sortante.
La fonction `pivot_dictionary` prend en entrée:
- `D` : le `np.array` de taille $(m+1, n+1)$ représentant le dictionnaire.
- `index_entering` : l'indice de la variable entrant dans le dictionnaire.
- `index_leaving` : l'indice de la variable sortante

Rappelons que lors de l'étape de pivot :
- la ligne pivot (d'indice `index_leaving + 1`) doit être ajoutée à toutes les lignes de `D` (à l'exeption de la ligne pivot) afin d'annuler les coefficients de la colonne pivot (d'indice `index_entering + 1`). 
- la ligne pivot doit être divisée par l'élément pivot `D[index_leaving + 1, index_entering + 1]`. Rappelons que par définition, l'élément pivot est strictement positif.

A la suite de toutes ces étapes la colonne `D[:,index_entering + 1]` doit être nulle sur tous les éléments à l'exeption de l'élément `index_leaving + 1` qui doit valoir `1`.
Vous pouvez utiliser une boucle pour réaliser l'étape de pivot.

In [ ]:
def pivot_dictionary(D,index_entering, index_leaving):
    nD = np.copy(D) # to force copy not just pointer copy
    # à coder
    return nD

**Test:** Vérifier que la fonction retourne bien le bon dictionnaire.

In [ ]:
index_entering = find_entering_variable(dict.D)
index_leaving = find_leaving_variable(dict.D, index_entering)
new_dict = pivot_dictionary(dict.D, index_entering, index_leaving)

print(new_dict)

### 3. Algorithme du simplexe



Dans cette section, nous allons implémenter l'algorithme du simplexe à partir d'un dictionnaire initial qui sera supposé valide, c'est à dire que la base du dictionnaire donne une solution de base admissible.
Ecrire la fonction `simplex_method` ci-dessous qui implémente l'algorithme du simplexe. Elle prend en entrée:
- `dict` : un `Dictionary` représentant le dictionnaire initial
- `display` : un booléen optionel qui permet de déclencher l'affichage du dictionnaire après chaque itération.

Elle retourne
- `optimal_cost` : la valeur de la fonction coût à l'optimum
- `solution` : un vecteur de taille $n$ représentant le sommet réalisant l'optimum
- `dict` : le dictionnaire à l'optimum

La fonction doit :
- émettre une erreur lorsque le programme n'est pas borné, c'est à dire lorsqu'à une itération donnée, aucune variable ne peut sortir du dictionnaire. Ceci peut s'effectuer à l'aide de `raise ValueError('Unbounded program')`.
- permuter les indices des variables entrantes et sortantes dans `dict.basic` afin de connaitre les variables dans la base et leur ordre dans le dictionnaire.
- retourner le minimiseur en tant que vecteur de taille $n$ contenant ainsi les variables de base et hors base (nulles)

Rappelons que l'algorithme a identifié le minimum lorsque tous les coûts réduits sont positifs ou nulles, autrement dit lorsqu'il n'y a plus de variable pouvant entrer dans la base.


In [4]:
def simplex_method(dict, display = True):

    # à coder 
    # penser à utiliser les lignes suivantes pour afficher le dictionnaire à chaque itération
    #    if display:
    #        dict.display()
    
    optimal_cost = ...
    solution = ...
    return optimal_cost, solution, dict


**Test:** Lancer l'algorithme du simplexe sur le dictionnaire et vérifier son exactitude.

In [ ]:
optimal_cost, solution, dict = simplex_method(dict)

## II. Méthode du simplexe en deux phases

Dans la section précédente, un algorithme du simplexe a été implémenté sous l'hypothèse de dictionnaire initial valide. Ce n'est pas le cas en général.

Cette section rassemble les fonctions permettant de définir `simplex_two_phases` résolvant le problème linéaire en deux étapes :
- la première qui permet de trouver un dictionnaire initial valide, en résolvant un problème linéaire auxiliaire pour lequel il existe une base initiale évidente admissible. Il est résolu à l'aide `simplex_method`
- la deuxième qui applique l'algorithme du simplexe sur ce dictionnaire initial.

Voir section 7.2 des notes de cours et Exercice 3.6.


In [ ]:
# la fonction suivante permet de vérifier si le dictionnaire est valide, c'est à dire si les variables sont positives
def is_valid_dictionary(dict):
    # all basic variables are non-negative
    test1 = np.all(dict.D[1:,0] >= 0)    
    # the matrix is invertible
    A = dict.D[1:,np.array(dict.basic)+1]
    test2 = np.linalg.cond(A) < 1e8
    return test1 and test2



# la fonction suivante créer le dictionnaire associé au problème linéaire auxiliaire à résoudre pour obtenir le dictionnaire intial
def make_auxiliary_dictionary(dict):
    m = dict.m
    n = dict.n

    aux_dict = Dictionary(m,n+m) 
    aux_dict.D[1:(m+1),1:(n+1)] = dict.D[1:,1:]
    
    # make sure b_i >= 0
    aux_dict.D[1:,0] = dict.D[1:,0]
    aux_dict.D[1:,:] *= np.sign(aux_dict.D[1:,0:1] + (aux_dict.D[1:,0:1] == 0))
    aux_dict.D[0,0] = -np.sum(aux_dict.D[1:,0]) 
    aux_dict.D[0,1:(n+1)] = -np.sum(aux_dict.D[1:,1:(n+1)], axis = 0) 

    aux_dict.D[1:(m+1),(n+1):] = np.eye(m)
    
    aux_dict.basic = list(range(n,n+m)) # basic variables are last variables
    return aux_dict


# cette fonction permet d'obtenir le dictionnaire initial en résolvant le problème auxiliaire et en récupérant les bonnes variables
# elle retourne None si il n'y a pas de dictionnaire intial, c'est à dire si l'ensemble des contraintes est vide.
def initial_phase(dict, display=True):
    aux_dict = make_auxiliary_dictionary(dict)
    
    if display:
        print('Auxiliary dictionary :')
        aux_dict.display()

    aux_optimal, aux_solution, aux_dict = simplex_method(aux_dict)
    nD = None

    if np.isclose(aux_optimal,0):
        nD = Dictionary(dict.m, dict.n)
        nD.c = dict.c
        nD.basic = aux_dict.basic 
        nD.D[0,0] = -np.sum( nD.c[nD.basic] * aux_dict.D[1:,0]) 
        nD.D[1:,:] = aux_dict.D[1:,:nD.n+1] 
        nD.D[0,1:] = nD.c - nD.c[nD.basic]@nD.D[1:,1:] # c - c_K^T A 

        nD.display()
    return nD

# cette fonction implémente l'algorithme du simplexe en deux phases
def simplex_two_phases(dict, display = True):
    if not is_valid_dictionary(dict):
        nD = initial_phase(dict, display)
        if nD == None:
            raise ValueError('Infeasible program')
    else:
        nD = dict

    if display:
        print('Initial dictionary:')
        nD.display()

    optimal_cost, solution, nD = simplex_method(nD, display) 
    return optimal_cost, solution, nD


On considère maintenant le programme linéaire suivant :

$$
\mathcal{P}=\left(\begin{array}{rrrlrlrl}
\min & z = & x_1 & + &  x_2 & \\
s.c. & & -3x_1 & - & 4x_2 &  \leq & -12 \\
& & 2x_1 &  +  & x_2 & \leq & 4 \\
& & x_j & \geq & 0 & \quad & j = & 1,2
\end{array}\right.
$$

Dont la représentation graphique est donnée ci-dessous:
    
![Problem 2](problem_2.png)
        
Le minimum est de $3$ et est atteint en $(3,0)$.

Le problème sous forme standard s'écrit :
$$
\mathcal{P}=\left(\begin{array}{rrrlrlrlrl}
\max & z = & - x_1 & - &  x_2 & & & \\
s.c. & & -3x_1 & - & 4x_2 & + & x_3 & = & -12 \\
& & 2x_1 &  +  & x_2 & + & x_4 & = & 4 \\
& & x_j & \geq & 0 & \quad & j = & 1,2,3,4
\end{array}\right.
$$

La base $K = \{x_3, x_4\}$ n'est donc pas une base réalisable, mais on peut tout de même 
écrire le dictionnaire pour cette base


**Todo:** Construire ce dictionnaire en python et lancer l'algorithme du simplexe en deux phases. Vérifier qu'il renvoie bien le minimum.

In [ ]:
n = #
m = #

dict = Dictionary(m,n+m)

dict.D[0,0] = #
dict.D[1:,0] = ##
dict.D[0,1:] = ##
dict.D[1:,1:] = ##
dict.basic = ##
dict.c[:] = ##

optimal_cost, solution, dict = simplex_two_phases(dict)

## III. Problème de transport

Dans cette section, nous allons appliquer l'algorithme du simplexe pour résoudre un problème de transport.


### 1. Le problème


Une brasserie dispose de deux usines de fabrication qui approvisionnent cinq bars. 
Au début de chaque semaine, chaque bar commande le volume de bière voulu pour la semaine au centre logistique de la brasserie qui choisit quelles usines fournissent quels bars.

La brasserie veut un moyen d'optimiser chaque semaine les coûts de transport de cette opération.

Supposons que pour cette semaine, l'usine A dispose de 1000L de bière et l'usine B dispose de 4000L et que les bars demandent 500L, 900L, 1800L, 200L et 700L chacun. 

Le schéma suivant représente graphiquement le problème :

![alt text](brewery_arcs.jpg)

### 2. Modélisation du problème

Les variables de decision représentent les volumes de bières (en L) livrés des usines aux bars (représentés par les flèches sur le schéma). Soient $U = \{a,b\}$ et $B = \{1,2,3,4,5\}$ et les variables :
$$
    x_{w,b} \geq 0, \quad \forall w \in W, b \in B
$$

Les contraintes viennent des besoins de chaque bar et des stocks de chaque usine. Le volume de bière distribué depuis une usine ne peut pas excéder son stock. On formule ainsi deux contraintes :
$$
    \sum_{b \in B} x_{w,b} \leq s_w, \quad \forall w \in W
$$
avec $s_a = 1000$ et $s_b = 4000$ représentant le stock de chaque usine.

Pour éviter les pertes de vente, le volume livré à un bar doit être superieur à sa demande qui peut stocker l'excèdent pour la semaine suivante. Ceci permet de formuler 5 contraintes :
$$
    \sum_{w \in W} x_{w,b} \geq d_b, \quad \forall b \in B
$$
avec $d_1 = 500$, $d_2 = 900$, $d_3 = 1800$, $d_4 = 200$, $d_5 = 700$ qui représentent les demandes de chaque bar.

Pour ce problème, nous supposerons que le coût de transport est proportionnel au volume transporté d'une usine à l'autre. Les coûts de transport entre les usine et les bars sont :
$$
\begin{array}{c | c c c c c}
    \hline
    & 1 & 2 & 3 & 4 & 5 \\ \hline
    a & 2 & 4 & 5 & 2 & 1 \\
    b & 3 & 1 & 3 & 2 & 3 \\
\end{array} = (c_{w,b})_{w \in W, b \in B}
$$
en euros par L de bière.
La fonction coût est donc 
$$
    \sum_{w \in W, b \in B} c_{w,b} x_{w,b}
$$




### 3. Ecriture du dictionnaire

Le problème linéaire contient $n = 10$ variables et $m = 7$ contraintes d'égalité et peut s'écrire sous la forme :
$$
    \min_{x \in \mathbb{R}^n} \langle c, x\rangle \quad \text{s.c.} \quad x \geq 0, A x \leq b
$$
avec :
- $x \in \mathbb{R}^{10}$ contenant les variables ordonnées sous cette forme $x = (x_{a,1}, x_{a,2}, x_{a,3}, x_{a,4}, x_{a,5}, x_{b,1}, x_{b,2}, x_{b,3}, x_{b,4}, x_{b,5})$
- $c \in \mathbb{R}^{10}$ contenant les couts pour chaque variable dans le meme ordre $c = (c_{a,1}, c_{a,2}, c_{a,3}, c_{a,4}, c_{a,5}, c_{b,1}, c_{b,2}, c_{b,3}, c_{b,4}, c_{b,5})$
- La matrice $A \in \mathbb{R}^{7,10}$
$$ 
A = 
\begin{pmatrix}
-1 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 & 0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & -1 & 0 \\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & -1 \\
1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1
\end{pmatrix}
$$
- Le vecteur $b = (-d_1, -d_2, -d_3, -d_4, -d_5, s_a, s_b ) = (-500, -900, -1800, -200, -700, 1000, 4000) $ 

**Todo:** Comprendre l'obtention du problème de transport sous forme canonique. Ecrire le problème en forme standard avec les 7 variables d'écarts $s_i \geq 0$ et écrire le dictionnaire associé. Puis compléter le code suivant afin de créér le dictionnaire dans la structure `Dictionary`. Les variables d'écarts seront placées après les variables du vecteur $x$.

In [ ]:
n = 10
m = 7

dict = Dictionary(m,n+m)

# création des labels
# les volumes en L
dict.labels = ['x_{%s,%d}' % (i,j) for i in ['a', 'b'] for j in range(0,5)]
# puis les variables d'écarts
[dict.labels.append('s_%d' % (i)) for i in range(0,7)]


dict.c[:] = ## 
A = np.zeros((m,n))
b = np.array(###)

dict.D[0,0] = #
dict.D[1:,0] = #
dict.D[0,1:] = #
dict.D[1:,1:(n+1)] = #
dict.D[1:,(n+1):] = #
dict.basic = list(range(n,n+m)) # les variables de bases sont à la fin

dict.display()

Comme le dictionnaire n'est pas valide (les variables d'écarts sont négatives) il faut résoudre ce problème à l'aide de la méthode `simplex_two_phases`.

**Todo:** résoudre le problème et analyser les résultats.

In [ ]:
optimal_cost, solution, dict = simplex_two_phases(dict)
